In [ ]:
import mysql.connector
import bcrypt
from tkinter import *
from tkinter import messagebox, ttk
import csv
from datetime import datetime

# --- MySQL Database Connection ---
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="pugazh",
    database="a1"
)
cursor = conn.cursor()

# Database and Table Initialization
cursor.execute("CREATE DATABASE IF NOT EXISTS a1")
cursor.execute("USE a1")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100),
    phone VARCHAR(15),
    address TEXT,
    account_creation_date DATE,
    total_spend DECIMAL(10, 2),
    loyalty_status VARCHAR(50),
    preferred_contact_method VARCHAR(50)
)""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Interactions (
    interaction_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    interaction_date DATE,
    notes TEXT,
    follow_up_date DATE,
    feedback_rating INT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
)""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(100) NOT NULL UNIQUE,
    password_hash VARCHAR(255) NOT NULL
)""")

# --- Helper Functions ---

def signup(username, password):
    if not username or not password:
        messagebox.showerror("Error", "Both fields are required.")
        return

    # Hash the password using bcrypt
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

    try:
        cursor.execute("INSERT INTO Users (username, password_hash) VALUES (%s, %s)", (username, hashed_password))
        conn.commit()
        messagebox.showinfo("Success", "Signup successful!")
        login_window()
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error: {err}")

def login(username, password):
    if not username or not password:
        messagebox.showerror("Error", "Both fields are required.")
        return

    cursor.execute("SELECT password_hash FROM Users WHERE username = %s", (username,))
    user = cursor.fetchone()
    
    if user and bcrypt.checkpw(password.encode('utf-8'), user[0].encode('utf-8')):
        messagebox.showinfo("Success", "Login successful!")
        open_main_menu()
        
    else:
        messagebox.showerror("Error", "Invalid username or password.")

def login_window():
    def cancel_login():
        login_win.destroy()

    login_win = Tk()
    login_win.title("Login")
    login_win.geometry("300x250")

    Label(login_win, text="Username").pack(pady=10)
    username_entry = Entry(login_win)
    username_entry.pack()

    Label(login_win, text="Password").pack(pady=10)
    password_entry = Entry(login_win, show="*")
    password_entry.pack()

    Button(login_win, text="Login", command=lambda: login(username_entry.get(), password_entry.get())).pack(pady=10)
    Button(login_win, text="Signup", command=lambda: signup_window()).pack(pady=5)
    Button(login_win, text="Cancel", command=cancel_login).pack(pady=5)

    login_win.mainloop()

def signup_window():
    def cancel_signup():
        signup_win.destroy()

    signup_win = Tk()
    signup_win.title("Signup")
    signup_win.geometry("300x250")

    Label(signup_win, text="Username").pack(pady=10)
    username_entry = Entry(signup_win)
    username_entry.pack()

    Label(signup_win, text="Password").pack(pady=10)
    password_entry = Entry(signup_win, show="*")
    password_entry.pack()

    Button(signup_win, text="Signup", command=lambda: signup(username_entry.get(), password_entry.get())).pack(pady=10)
    Button(signup_win, text="Cancel", command=cancel_signup).pack(pady=5)

    signup_win.mainloop()

# --- Main Menu GUI ---
def open_main_menu():
    main_menu = Tk()
    main_menu.title("CRM Tool - Main Menu")
    main_menu.geometry("990x900")

    # Create Notebook (tabs)
    notebook = ttk.Notebook(main_menu)
    notebook.pack(fill=BOTH, expand=YES)

    # Customer Management Tab
    customer_frame = Frame(notebook)
    notebook.add(customer_frame, text="Manage Customers")

    Label(customer_frame, text="Add Customer", font=("Arial", 14)).pack(pady=5)
    Label(customer_frame, text="Name").pack()
    name_entry = Entry(customer_frame)
    name_entry.pack()

    Label(customer_frame, text="Email").pack()
    email_entry = Entry(customer_frame)
    email_entry.pack()

    Label(customer_frame, text="Phone").pack()
    phone_entry = Entry(customer_frame)
    phone_entry.pack()

    Label(customer_frame, text="Address").pack()
    address_entry = Entry(customer_frame)
    address_entry.pack()

    Label(customer_frame, text="Account Creation Date (YYYY-MM-DD)").pack()
    account_creation_date_entry = Entry(customer_frame)
    account_creation_date_entry.pack()

    Label(customer_frame, text="Total Spend").pack()
    total_spend_entry = Entry(customer_frame)
    total_spend_entry.pack()

    Label(customer_frame, text="Loyalty Status").pack()
    loyalty_status_entry = Entry(customer_frame)
    loyalty_status_entry.pack()

    Label(customer_frame, text="Preferred Contact Method").pack()
    preferred_contact_method_entry = Entry(customer_frame)
    preferred_contact_method_entry.pack()

    Button(customer_frame, text="Add Customer", command=lambda: add_customer(
        name_entry.get(), email_entry.get(), phone_entry.get(), address_entry.get(),
        account_creation_date_entry.get(), total_spend_entry.get(),
        loyalty_status_entry.get(), preferred_contact_method_entry.get())).pack(pady=10)

    Label(customer_frame, text="Customer List", font=("Arial", 14)).pack(pady=10)

    search_entry = Entry(customer_frame)
    search_entry.pack()
    Button(customer_frame, text="Search", command=lambda: search_customer(search_entry.get())).pack(pady=5)
    Button(customer_frame, text="Show All Customers", command=show_all_customers).pack(pady=5)
    Button(customer_frame, text="Export to CSV", command=export_customers).pack(pady=5)
    
    global customer_table
    customer_table = ttk.Treeview(customer_frame, columns=("ID", "Name", "Email", "Phone", "Address", "Account Creation Date","Total Spend", "Loyalty Status", "Preferred Contact Method"), show="headings")
    customer_table.heading("ID", text="ID")
    customer_table.heading("Name", text="Name")
    customer_table.heading("Email", text="Email")
    customer_table.heading("Phone", text="Phone")
    customer_table.heading("Address", text="Address")
    customer_table.heading("Account Creation Date", text="Account Creation Date")
    # customer_table.heading("Last Purchase Date", text="Last Purchase Date")
    customer_table.heading("Total Spend", text="Total Spend")
    customer_table.heading("Loyalty Status", text="Loyalty Status")
    customer_table.heading("Preferred Contact Method", text="Preferred Contact Method")
    customer_table.pack(fill=BOTH, expand=YES)

    # Interaction Management Tab
    interaction_frame = Frame(notebook)
    notebook.add(interaction_frame, text="Manage Interactions")
    
    Label(interaction_frame, text="Add Interaction", font=("Arial", 14)).pack(pady=10)

    Label(interaction_frame, text="Customer ID").pack()
    customer_id_entry = Entry(interaction_frame)
    customer_id_entry.pack()

    Label(interaction_frame, text="Interaction Date").pack()
    interaction_date_entry = Entry(interaction_frame)  # Use Entry widget for date input
    interaction_date_entry.pack()

    Label(interaction_frame, text="Notes").pack()
    notes_entry = Entry(interaction_frame)
    notes_entry.pack()

    Label(interaction_frame, text="Follow-up Date").pack()
    follow_up_date_entry = Entry(interaction_frame)  # Use Entry widget for date input
    follow_up_date_entry.pack()

    Label(interaction_frame, text="Feedback/Rating (1-5)").pack()
    feedback_rating_entry = Entry(interaction_frame)
    feedback_rating_entry.pack()

    Button(interaction_frame, text="Add Interaction", command=lambda: add_interaction(
        customer_id_entry.get(), interaction_date_entry.get(), notes_entry.get(),
        follow_up_date_entry.get(), feedback_rating_entry.get())).pack(pady=10)

    Label(interaction_frame, text="Interaction History", font=("Arial", 14)).pack(pady=10)

    Button(interaction_frame, text="Show All Interactions", command=show_all_interactions).pack(pady=5)

    # --- Search Interaction by Customer ID ---
    Label(interaction_frame, text="Search Interaction by Customer ID").pack(pady=5)
    search_customer_id_entry = Entry(interaction_frame)
    search_customer_id_entry.pack()

    Button(interaction_frame, text="Search", command=lambda: search_interaction_by_customer_id(search_customer_id_entry.get())).pack(pady=5)


    global interaction_table
    interaction_table = ttk.Treeview(interaction_frame, columns=("ID", "Customer ID", "Date", "Notes", "Follow-up Date", "Feedback/Rating"), show="headings")
    interaction_table.heading("ID", text="ID")
    interaction_table.heading("Customer ID", text="Customer ID")
    interaction_table.heading("Date", text="Date")
    interaction_table.heading("Notes", text="Notes")
    interaction_table.heading("Follow-up Date", text="Follow-up Date")
    interaction_table.heading("Feedback/Rating", text="Feedback/Rating")
    interaction_table.pack(fill=BOTH, expand=YES)

    main_menu.mainloop()



def add_customer(name, email, phone, address, account_creation_date, total_spend, loyalty_status, preferred_contact_method):
    try:
        # Ensure the date is in the correct format
        account_creation_date = datetime.strptime(account_creation_date, "%Y-%m-%d").date()
        
        
        cursor.execute("INSERT INTO Customers (name, email, phone, address, account_creation_date, total_spend, loyalty_status, preferred_contact_method) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                       (name, email, phone, address, account_creation_date, total_spend, loyalty_status, preferred_contact_method))
        conn.commit()
        messagebox.showinfo("Success", "Customer added successfully!")
        show_all_customers()
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error: {err}")
    except ValueError as e:
        messagebox.showerror("Error", "Invalid date format. Please use YYYY-MM-DD.")


def show_all_customers():
    for row in customer_table.get_children():
        customer_table.delete(row)
    
    cursor.execute("SELECT * FROM Customers")
    customers = cursor.fetchall()
    
    for customer in customers:
        customer_table.insert("", "end", values=customer)

def search_customer(search_term):
    for row in customer_table.get_children():
        customer_table.delete(row)

    cursor.execute("SELECT * FROM Customers WHERE name LIKE %s OR email LIKE %s OR phone LIKE %s", 
                   ('%' + search_term + '%', '%' + search_term + '%', '%' + search_term + '%'))
    customers = cursor.fetchall()

    for customer in customers:
        customer_table.insert("", "end", values=customer)

def export_customers():
    customers = customer_table.get_children()
    with open("customers.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([col for col in customer_table["columns"]])
        for customer in customers:
            writer.writerow(customer_table.item(customer)["values"])

def add_interaction(customer_id, interaction_date, notes, follow_up_date, feedback_rating):
    try:
        cursor.execute("INSERT INTO Interactions (customer_id, interaction_date, notes, follow_up_date, feedback_rating) VALUES (%s, %s, %s, %s, %s)",
                       (customer_id, interaction_date, notes, follow_up_date, feedback_rating))
        conn.commit()
        messagebox.showinfo("Success", "Interaction added successfully!")
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error: {err}")

def show_all_interactions():
    for row in interaction_table.get_children():
        interaction_table.delete(row)

    cursor.execute("SELECT * FROM Interactions")
    interactions = cursor.fetchall()

    for interaction in interactions:
        interaction_table.insert("", "end", values=interaction)
# --- Search Interaction by Customer ID ---
def search_interaction_by_customer_id(customer_id):
    # Clear the current table
    for row in interaction_table.get_children():
        interaction_table.delete(row)

    try:
        cursor.execute("SELECT * FROM Interactions WHERE customer_id = %s", (customer_id,))
        interactions = cursor.fetchall()

        if interactions:
            for interaction in interactions:
                interaction_table.insert("", "end", values=interaction)
        else:
            messagebox.showinfo("No Results", "No interactions found for the given Customer ID.")
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error: {err}")

login_window()


In [ ]:






++